In [1]:
from IPython.display import display,HTML,clear_output
import json

# Read explanation report

In [2]:
explanation_report_filepath = "/Users/salvatore/PycharmProjects/T-EBAnO-Express/outputs/20201117_bert_model_imdb_reviews_exp_0/local_explanations_experiments/20201204_105147/local_explanations/local_explanation_report_0.json"


In [3]:
with open(explanation_report_filepath) as json_file:
    explanation_report = json.load(json_file)

# Parse explanation report

In [5]:
input_info = explanation_report["input_info"]
input_positions_tokens = input_info["positions_tokens"]
raw_text = input_info["raw_text"]
cleaned_text = input_info["cleaned_text"]
preprocessed_text = input_info["preprocessed_text"]

In [6]:
pos_local_explanations = []  # pos features without combinations
sen_local_explanations = []  # sen features without combinations
mlwe_local_explanations = []  # mlwe features without combinations
pos_local_explanations_comb = []  # All pos features (also combinations)
sen_local_explanations_comb = []  # All sen features (also combinations)
mlwe_local_explanations_comb = []  # All mlwe features (also combinations)

for le in explanation_report["local_explanations"]:
    if le["feature_type"] == "POS" and le["combination"] == 1:
        pos_local_explanations.append(le)
    if le["feature_type"] == "SEN" and le["combination"] == 1:
        sen_local_explanations.append(le)
    if le["feature_type"] == "MLWE" and le["combination"] == 1:
        mlwe_local_explanations.append(le)
    if le["feature_type"] == "POS":
        pos_local_explanations_comb.append(le)
    if le["feature_type"] == "SEN":  
        sen_local_explanations_comb.append(le)
    if le["feature_type"] == "MLWE":
        mlwe_local_explanations_comb.append(le)
        

In [7]:
pos_positions_tokens_score = {} # Dictionary for pos features with position as key and as value the tuple (token, nPIR)
sen_positions_tokens_score = {}#  Dictionary for sen features with position as key and as value the tuple (token, nPIR)
mlwe_positions_tokens_score = {} # Dictionary for mlwe features with position as key and as value the tuple (token, nPIR)

for le in pos_local_explanations:
    for position, token in le["positions_tokens"].items():
        pos_position_token_score[position] = (token, round(le["nPIR_original_top_class"],4))
        
for le in sen_local_explanations:
    for position, token in le["positions_tokens"].items():
        sen_position_token_score[position] = (token, round(le["nPIR_original_top_class"],4))

for le in mlwe_local_explanations:
    for position, token in le["positions_tokens"].items():
        mlwe_position_token_score[position] = (token, round(le["nPIR_original_top_class"],4))
    

# Utility functions


In [8]:
def label_name(label):
    return ["Negative","Positive"][label]

def get_summary_string(position_token_score):
    positive_color = "background-color:#00FF00"
    negative_color = "background-color:#FF00FF"
    html_string = ""

    for position in sorted(position_token_score.keys(), key=lambda k: int(k)):
        token_score = position_token_score[position]
        if token_score[1] >= 0:
            positive_color = "background-color:rgba(124, 252, 0, {})".format(token_score[1])
            html_string = html_string + '<span style="{}"><b>{}</b></span> '.format(positive_color,token_score[0])
        else:
            negative_color = "background-color:rgba(255, 99, 71, {})".format(token_score[1])
            html_string = html_string + '<span style="{}"><b>{}</b></span> '.format(negative_color,token_score[0])
            
    return html_string

def get_feature_colored_html_string(input_positions_tokens, feature_positions_tokens):
    html_string = ""
    feature_color = "background-color:rgba(0, 255, 255, 1)"
    for position in range(len(input_positions_tokens)):
        if str(position) in feature_positions_tokens:
            html_string = html_string + '<span style="{}"><b>{}</b></span> '.format(feature_color,input_positions_tokens[position])
        else:
            html_string = html_string + '<span><b>{}</b></span> '.format(input_positions_tokens[position])
    return html_string
    

def print_local_explanation(le, input_positions_tokens):
    if len(le["position_token"]) == 0:
        return
    print("LocalExplanationId: {}, FeatureId: {}".format(le["local_explanation_id"], le["feature_id"]))
    print("feature type: ",le["feature_type"])
    print("feature description: ", le["feature_description"])
    print("feature (position,token): " ,le["positions_tokens"])
    print("Perturbed Text:  ",le["perturbed_text"])
    print("perturbed probabilities: ",le["perturbed_probabilities"])
    print("nPIR Original Class: " ,round(le["nPIR_original_top_class"],3))
    display(HTML(get_feature_colored_html_string(input_position_tokens,le["positions_tokens"])))
    print("\n")
    print("\n")
    return

In [9]:
display(HTML('<h1>Input Text info</h1>'))
display(HTML('Raw Text:   <b>{} </b>'.format(raw_text)))
display(HTML('Clean Text:   <b>{} </b>'.format(cleaned_text)))
display(HTML('Preprocessed Text:   <b>{} </b>'.format(preprocessed_text)))

In [10]:
display(HTML('<h1>Multi-Layer Word Embedding Features Summary</h1>'))
display(HTML('Predicted Class: <b>{} </b>'.format(label_name(input_info["original_label"]))))
display(HTML('Original Probability: <b>{} </b>'.format(input_info["original_probabilities"])))

html_string = get_summary_string(mlwe_position_token_score)
        
display(HTML(html_string))

display(HTML('(Features are highlighted in green if are positively influential for the predicted class, red otherwise. The intensity of the color is proportional to the nPIR score obtained by the word.) '))  


# MLWE Most influential feature

Feature with highest nPIR 

In [12]:
most_influnetial_mlwe_feature =max(mlwe_local_explanations_comb, key= lambda le: le["nPIR_original_top_class"])
print_local_explanation(most_influnetial_mlwe_feature,input_positions_tokens)

KeyError: 'position_token'

<h2>Query Multi-Layer Word Embedding features</h2>

In [140]:
print("Original Probabilities: {}".format(input_info["original_probabilities"]))
print("Original label: {}".format(input_info["original_label"]))
print("\n\n")
min_nPIR_mlwe = -1
max_nPIR_mlwe = 1

for le in sorted(mlwe_local_explanations_comb, key=lambda le: le["nPIR_original_top_class"], reverse=True ):
    if le["nPIR_original_top_class"] >= min_nPIR_mlwe and le["nPIR_original_top_class"] <= max_nPIR_mlwe:
        print_local_explanation(le, input_positions_tokens)
        

Original Probabilities: [0.9941796660423279, 0.005820333957672119]
Original label: 0



LocalExplanationId: 43, FeatureId: 9
feature type:  MLWE
feature description:  Combination of Cluster 1 and Cluster 2
feature (position,token):  {'0': 'the', '7': 'film', '8': 'instead', '10': 'not', '11': 'as', '18': 'expected', '19': 'more', '20': 'for', '21': 'this', '22': 'movie', '23': '.', '6': 'the', '9': 'is', '17': 'i'}
Perturbed Text:   book is very nice. good as the book.
perturbed probabilities:  [0.0017904043197631836, 0.9982095956802368]
nPIR Original Class:  0.998






LocalExplanationId: 35, FeatureId: 1
feature type:  MLWE
feature description:  Cluster 1
feature (position,token):  {'0': 'the', '7': 'film', '8': 'instead', '10': 'not', '11': 'as', '18': 'expected', '19': 'more', '20': 'for', '21': 'this', '22': 'movie', '23': '.'}
Perturbed Text:   book is very nice. the is good as the book. i
perturbed probabilities:  [0.0025101304054260254, 0.997489869594574]
nPIR Original Class:  0.997






LocalExplanationId: 45, FeatureId: 11
feature type:  MLWE
feature description:  Combination of Cluster 1 and Cluster 4
feature (position,token):  {'0': 'the', '7': 'film', '8': 'instead', '10': 'not', '11': 'as', '18': 'expected', '19': 'more', '20': 'for', '21': 'this', '22': 'movie', '23': '.', '2': 'is', '16': '.'}
Perturbed Text:   book very nice. the is good as the book i
perturbed probabilities:  [0.002787947654724121, 0.9972120523452759]
nPIR Original Class:  0.997






LocalExplanationId: 44, FeatureId: 10
feature type:  MLWE
feature description:  Combination of Cluster 1 and Cluster 3
feature (position,token):  {'0': 'the', '7': 'film', '8': 'instead', '10': 'not', '11': 'as', '18': 'expected', '19': 'more', '20': 'for', '21': 'this', '22': 'movie', '23': '.', '12': 'good', '13': 'as', '14': 'the'}
Perturbed Text:   book is very nice. the is book. i
perturbed probabilities:  [0.003794372081756592, 0.9962056279182434]
nPIR Original Class:  0.996






LocalExplanationId: 39, FeatureId: 5
feature type:  MLWE
feature description:  Combination of Cluster 0 and Cluster 1
feature (position,token):  {'1': 'book', '3': 'very', '4': 'nice', '5': '.', '15': 'book', '0': 'the', '7': 'film', '8': 'instead', '10': 'not', '11': 'as', '18': 'expected', '19': 'more', '20': 'for', '21': 'this', '22': 'movie', '23': '.'}
Perturbed Text:   is the is good as the. i
perturbed probabilities:  [0.024663448333740234, 0.9753365516662598]
nPIR Original Class:  0.975


...





LocalExplanationId: 47, FeatureId: 13
feature type:  MLWE
feature description:  Combination of Cluster 2 and Cluster 4
feature (position,token):  {'6': 'the', '9': 'is', '17': 'i', '2': 'is', '16': '.'}
Perturbed Text:   the book very nice. film instead not as good as the book expected more for this movie.
perturbed probabilities:  [0.9910808801651001, 0.008919119834899902]
nPIR Original Class:  0.006






LocalExplanationId: 48, FeatureId: 14
feature type:  MLWE
feature description:  Combination of Cluster 3 and Cluster 4
feature (position,token):  {'12': 'good', '13': 'as', '14': 'the', '2': 'is', '16': '.'}
Perturbed Text:   the book very nice. the film instead is not as book i expected more for this movie.
perturbed probabilities:  [0.992160439491272, 0.007839560508728027]
nPIR Original Class:  0.004






LocalExplanationId: 46, FeatureId: 12
feature type:  MLWE
feature description:  Combination of Cluster 2 and Cluster 3
feature (position,token):  {'6': 'the', '9': 'is', '17': 'i', '12': 'good', '13': 'as', '14': 'the'}
Perturbed Text:   the book is very nice. film instead not as book. expected more for this movie.
perturbed probabilities:  [0.9925110340118408, 0.007488936185836792]
nPIR Original Class:  0.003






LocalExplanationId: 37, FeatureId: 3
feature type:  MLWE
feature description:  Cluster 3
feature (position,token):  {'12': 'good', '13': 'as', '14': 'the'}
Perturbed Text:   the book is very nice. the film instead is not as book. i expected more for this movie.
perturbed probabilities:  [0.995774507522583, 0.0042254626750946045]
nPIR Original Class:  -0.003






LocalExplanationId: 36, FeatureId: 2
feature type:  MLWE
feature description:  Cluster 2
feature (position,token):  {'6': 'the', '9': 'is', '17': 'i'}
Perturbed Text:   the book is very nice. film instead not as good as the book. expected more for this movie.
perturbed probabilities:  [0.9970859289169312, 0.00291404128074646]
nPIR Original Class:  -0.006


In [141]:
display(HTML('<h1>Parts-of-speech Features Summary</h1>'))
display(HTML('<b>Predicted Class: {} </b>'.format(label_name(input_info["original_label"]))))
display(HTML('<b>Original Probability: {} </b>'.format(input_info["original_probabilities"])))

html_string = get_summary_string(pos_position_token_score)
display(HTML(html_string))

display(HTML('(Features are highlighted in green if are positively influential for the predicted class, red otherwise. The intensity of the color is proportional to the nPIR score obtained by the word.) '))  


# POS Most influential feature

In [142]:
most_influnetial_pos_feature =max(pos_local_explanations_comb, key= lambda le: le["nPIR_original_top_class"])
print_local_explanation(most_influnetial_pos_feature,input_position_tokens)

LocalExplanationId: 3, FeatureId: 3
feature type:  POS
feature description:  Adverbs, Modals
feature (position,token):  {'3': 'very', '8': 'instead', '10': 'not', '11': 'as'}
Perturbed Text:   the book is nice. the film is good as the book. i expected more for this movie.
perturbed probabilities:  [0.001850426197052002, 0.998149573802948]
nPIR Original Class:  0.998


<h1>Query Parts-of-speech Features</h1>

In [143]:
print("Original Probabilities: {}".format(input_info["original_probabilities"]))
print("Original label: {}".format(input_info["original_label"]))
print("\n\n")
min_nPIR_pos = -1
max_nPIR_pos = 1

for le in sorted(pos_local_explanations_comb, key=lambda le: le["nPIR_original_top_class"], reverse=True ):
    if le["nPIR_original_top_class"] >= min_nPIR_pos and le["nPIR_original_top_class"] <= max_nPIR_pos:
        print_local_explanation(le, input_position_tokens)

Original Probabilities: [0.9941796660423279, 0.005820333957672119]
Original label: 0



LocalExplanationId: 3, FeatureId: 3
feature type:  POS
feature description:  Adverbs, Modals
feature (position,token):  {'3': 'very', '8': 'instead', '10': 'not', '11': 'as'}
Perturbed Text:   the book is nice. the film is good as the book. i expected more for this movie.
perturbed probabilities:  [0.001850426197052002, 0.998149573802948]
nPIR Original Class:  0.998






LocalExplanationId: 22, FeatureId: 22
feature type:  POS
feature description:  Combination of Adverbs, Modals and Pronouns, Predeterminers, WH
feature (position,token):  {'3': 'very', '8': 'instead', '10': 'not', '11': 'as'}
Perturbed Text:   the book is nice. the film is good as the book. i expected more for this movie.
perturbed probabilities:  [0.001850426197052002, 0.998149573802948]
nPIR Original Class:  0.998






LocalExplanationId: 23, FeatureId: 23
feature type:  POS
feature description:  Combination of Adverbs, Modals and Interjections
feature (position,token):  {'3': 'very', '8': 'instead', '10': 'not', '11': 'as'}
Perturbed Text:   the book is nice. the film is good as the book. i expected more for this movie.
perturbed probabilities:  [0.001850426197052002, 0.998149573802948]
nPIR Original Class:  0.998






LocalExplanationId: 24, FeatureId: 24
feature type:  POS
feature description:  Combination of Adverbs, Modals and Others (Cardinal Numbers, Foreign Words, Symbols)
feature (position,token):  {'3': 'very', '8': 'instead', '10': 'not', '11': 'as'}
Perturbed Text:   the book is nice. the film is good as the book. i expected more for this movie.
perturbed probabilities:  [0.001850426197052002, 0.998149573802948]
nPIR Original Class:  0.998






LocalExplanationId: 18, FeatureId: 18
feature type:  POS
feature description:  Combination of Verbs and Adverbs, Modals
feature (position,token):  {'2': 'is', '9': 'is', '18': 'expected', '3': 'very', '8': 'instead', '10': 'not', '11': 'as'}
Perturbed Text:   the book nice. the film good as the book. i more for this movie.
perturbed probabilities:  [0.0022974014282226562, 0.9977025985717773]
nPIR Original Class:  0.998


...





LocalExplanationId: 17, FeatureId: 17
feature type:  POS
feature description:  Combination of Nouns and Others (Cardinal Numbers, Foreign Words, Symbols)
feature (position,token):  {'1': 'book', '7': 'film', '15': 'book', '17': 'i', '22': 'movie'}
Perturbed Text:   the is very nice. the instead is not as good as the. expected more for this.
perturbed probabilities:  [0.9224529266357422, 0.0775471031665802]
nPIR Original Class:  0.126






LocalExplanationId: 0, FeatureId: 0
feature type:  POS
feature description:  Adjectives
feature (position,token):  {'4': 'nice', '12': 'good', '19': 'more'}
Perturbed Text:   the book is very. the film instead is not as as the book. i expected for this movie.
perturbed probabilities:  [0.9811475872993469, 0.018852412700653076]
nPIR Original Class:  0.025






LocalExplanationId: 10, FeatureId: 10
feature type:  POS
feature description:  Combination of Adjectives and Pronouns, Predeterminers, WH
feature (position,token):  {'4': 'nice', '12': 'good', '19': 'more'}
Perturbed Text:   the book is very. the film instead is not as as the book. i expected for this movie.
perturbed probabilities:  [0.9811475872993469, 0.018852412700653076]
nPIR Original Class:  0.025






LocalExplanationId: 11, FeatureId: 11
feature type:  POS
feature description:  Combination of Adjectives and Interjections
feature (position,token):  {'4': 'nice', '12': 'good', '19': 'more'}
Perturbed Text:   the book is very. the film instead is not as as the book. i expected for this movie.
perturbed probabilities:  [0.9811475872993469, 0.018852412700653076]
nPIR Original Class:  0.025






LocalExplanationId: 12, FeatureId: 12
feature type:  POS
feature description:  Combination of Adjectives and Others (Cardinal Numbers, Foreign Words, Symbols)
feature (position,token):  {'4': 'nice', '12': 'good', '19': 'more'}
Perturbed Text:   the book is very. the film instead is not as as the book. i expected for this movie.
perturbed probabilities:  [0.9811475872993469, 0.018852412700653076]
nPIR Original Class:  0.025


In [144]:
display(HTML('<h1>Sentences Features Summary</h1>'))
display(HTML('<b>Predicted Class: {} </b>'.format(label_name(input_info["original_label"]))))
display(HTML('<b>Original Probability: {} </b>'.format(input_info["original_probabilities"])))

html_sen_string = get_summary_string(sen_position_token_score)
        
display(HTML(html_sen_string))

display(HTML('(Features are highlighted in green if are positively influential for the predicted class, red otherwise. The intensity of the color is proportional to the nPIR score obtained by the word.) '))  


# SEN Most influential feature

In [145]:
most_influnetial_sen_feature =max(sen_local_explanations_comb, key= lambda le: le["nPIR_original_top_class"])
print_local_explanation(most_influnetial_sen_feature,input_positions_tokens)

LocalExplanationId: 33, FeatureId: 5
feature type:  SEN
feature description:  Combination of 2° Sentence and 3° Sentence
feature (position,token):  {'6': 'the', '7': 'film', '8': 'instead', '9': 'is', '10': 'not', '11': 'as', '12': 'good', '13': 'as', '14': 'the', '15': 'book', '16': '.', '17': 'i', '18': 'expected', '19': 'more', '20': 'for', '21': 'this', '22': 'movie', '23': '.'}
Perturbed Text:   the book is very nice.
perturbed probabilities:  [0.001116335391998291, 0.9988836646080017]
nPIR Original Class:  0.999


<h1> Query Sentences Features</h1>

In [146]:
print("Original Probabilities: {}".format(input_info["original_probabilities"]))
print("Original label: {}".format(input_info["original_label"]))
print("\n\n")
min_nPIR_sen = -1
max_nPIR_sen = 1

for le in sorted(sen_local_explanations_comb, key=lambda le: le["nPIR_original_top_class"], reverse=True ):
    if le["nPIR_original_top_class"] >= min_nPIR_sen and le["nPIR_original_top_class"] <= max_nPIR_sen:
        print_local_explanation(le, input_positions_tokens)

Original Probabilities: [0.9941796660423279, 0.005820333957672119]
Original label: 0



LocalExplanationId: 33, FeatureId: 5
feature type:  SEN
feature description:  Combination of 2° Sentence and 3° Sentence
feature (position,token):  {'6': 'the', '7': 'film', '8': 'instead', '9': 'is', '10': 'not', '11': 'as', '12': 'good', '13': 'as', '14': 'the', '15': 'book', '16': '.', '17': 'i', '18': 'expected', '19': 'more', '20': 'for', '21': 'this', '22': 'movie', '23': '.'}
Perturbed Text:   the book is very nice.
perturbed probabilities:  [0.001116335391998291, 0.9988836646080017]
nPIR Original Class:  0.999






LocalExplanationId: 29, FeatureId: 1
feature type:  SEN
feature description:  2° Sentence
feature (position,token):  {'6': 'the', '7': 'film', '8': 'instead', '9': 'is', '10': 'not', '11': 'as', '12': 'good', '13': 'as', '14': 'the', '15': 'book', '16': '.'}
Perturbed Text:   the book is very nice. i expected more for this movie.
perturbed probabilities:  [0.03649163246154785, 0.9635083675384521]
nPIR Original Class:  0.963






LocalExplanationId: 32, FeatureId: 4
feature type:  SEN
feature description:  Combination of 1° Sentence and 3° Sentence
feature (position,token):  {'0': 'the', '1': 'book', '2': 'is', '3': 'very', '4': 'nice', '5': '.', '17': 'i', '18': 'expected', '19': 'more', '20': 'for', '21': 'this', '22': 'movie', '23': '.'}
Perturbed Text:   the film instead is not as good as the book.
perturbed probabilities:  [0.8644679188728333, 0.13553208112716675]
nPIR Original Class:  0.208






LocalExplanationId: 31, FeatureId: 3
feature type:  SEN
feature description:  Combination of 1° Sentence and 2° Sentence
feature (position,token):  {'0': 'the', '1': 'book', '2': 'is', '3': 'very', '4': 'nice', '5': '.', '6': 'the', '7': 'film', '8': 'instead', '9': 'is', '10': 'not', '11': 'as', '12': 'good', '13': 'as', '14': 'the', '15': 'book', '16': '.'}
Perturbed Text:   i expected more for this movie.
perturbed probabilities:  [0.8858973383903503, 0.11410266160964966]
nPIR Original Class:  0.179






LocalExplanationId: 28, FeatureId: 0
feature type:  SEN
feature description:  1° Sentence
feature (position,token):  {'0': 'the', '1': 'book', '2': 'is', '3': 'very', '4': 'nice', '5': '.'}
Perturbed Text:   the film instead is not as good as the book. i expected more for this movie.
perturbed probabilities:  [0.9610356092453003, 0.038964420557022095]
nPIR Original Class:  0.062






LocalExplanationId: 30, FeatureId: 2
feature type:  SEN
feature description:  3° Sentence
feature (position,token):  {'17': 'i', '18': 'expected', '19': 'more', '20': 'for', '21': 'this', '22': 'movie', '23': '.'}
Perturbed Text:   the book is very nice. the film instead is not as good as the book.
perturbed probabilities:  [0.9878309965133667, 0.012169033288955688]
nPIR Original Class:  0.013
